In [32]:
import os
import sys
import subprocess
import time

In [2]:
# 执行中断命令 返回结果
def bashInfo(commond):
    return os.popen(commond).read().strip('\n')

## 检查设备在线情况
adb devices

In [103]:
# 检查设备在线
def checkDevice():
    devices = os.popen('adb devices').read().strip('\n').split('\n')
    if len(devices) > 1:
        return devices[1].split('\t')[0]
    else:
        return ""

## 打开APP
使用adb shell am start -n com.android.settings/com.tts.trip.WelcomeActivity
使用app包与要启动的Activity的全路径名称

In [4]:
# 打开APP
def openApp(package, name):
    appCommond = 'adb shell am start -n %s/%s' % (package, name)
    info = os.popen(appCommond).read().strip('\n')
    if 'Error' in info:
        return False
    else:
        return True

## 获取前台Activity

adb shell dumpsys activity activities | sed -En -e '/Running activities/,/Run #0/p'

adb shell dumpsys activity | grep -i run

In [5]:
# 获取前台Activity
def getFrontActivity(package):
    info = ''
    activitys = []
    if package.strip('\n') == '':
        info = os.popen('adb shell dumpsys activity activities | sed -En -e \'/Running activities/,/Run #0/p\'').read().strip('\n')
    else:
        info = os.popen('adb shell dumpsys activity activities | sed -En -e \'/Running activities/,/Run #0/p\' | grep %s' % package).read().strip('\n')
    
    # 换行分组
    infos = info.split('\n')
    for one in infos:
        if 'ActivityRecord' in one:
            words = one.split(' ')
            for word in words:
                if '/' in word:
                    activitys.append(word)
    return activitys

## 获取手机上已安装的应用

- 不使用参数：adb shell pm list packages，打印设备/模拟器上的所有软件包
- 使用-f参数：adb shell pm list packages -f,输出包和包相关联的文件
- 使用-d参数：adb shell pm list packages -d,只输出禁用的包。由于本机禁用没有，输出为空。
- 使用-e参数：adb shell pm list packages -e,只输出启用的包。
- 使用-s参数：adb shell pm list packages -s,只输出系统的包。
- 使用-3参数：adb shell pm list packages -3,只输出第三方的包。
- 使用-i参数：adb shell pm list packages -i,只输出包和安装信息（安装来源）。
- 使用-u参数：adb shell pm list packages -u,只输出包和未安装包信息（安装来源）
- 使用--user参数：adb shell pm list packages --user <USER_ID>，根据用户id查询用户的空间的所有包，USER_ID代表当前连接设备的顺序，从零开始：
- 使用-e参数且设置过滤条件：adb shell pm list packages -e "ximalaya",只输出启用的包。

In [58]:
# 获取第三方应用
def getThirdApps():
    packageNames = []
    commond = 'adb shell pm list packages -3'
    info = os.popen(commond).read().strip('\n')
    apps = info.split('\n')
    for app in apps:
        app = app.replace('package:', '')
        packageNames.append(app)
    return packageNames

## 把apk包从手机下载到本地
### 根据包名获取apk位置
adb shell pm path 包名
### 拉取
adb pull 手机中apk的位置 保存在本地的位置

In [101]:
# 拉取apk
def pullApk(package):
    getPathCommond = 'adb shell pm path %s' % package
    path = bashInfo(getPathCommond)
    path = path.replace('package:', '')
    pullCommond = 'adb pull %s apk/%s.apk' % (path, package)
    # 持续输出
    subp = subprocess.Popen(pullCommond,shell=True,stdout=subprocess.PIPE)
    c=subp.stdout.readline()
    str = ''
    while c:
        str = c.decode().replace('\n','')
        print(str,end='\r')
        c=subp.stdout.readline()
    print('')
    print('\033[1;31m拉取%s完成\033[0m' % app)
    #return subp.returncode

In [ ]:
# 获取设备名称
#device = checkDevice()
#print('查找到设备%s' % device)
#startStatus = openApp('com.tts.hybird', 'com.tts.trip.WelcomeActivity')
# if startStatus:
#     print('启动app成功')
# else:
#     print('启动app失败')
#activitys = getFrontActivity('')
#print(activitys)
apps = getThirdApps()
for app in apps:
    pullApk(app)
